In [1]:
#生成矩形数据的函数
def get_rectangle():
    import random

    #随机得到矩形的宽和高,值域0-1之间的小数
    width = random.random()
    height = random.random()

    #计算面积
    s = width * height

    return width, height, s


get_rectangle()

(0.5184137805253416, 0.26786610058702154, 0.13886547787989925)

In [2]:
import torch


#定义数据集
class Dataset(torch.utils.data.Dataset):

    #正常应该在这里执行数据的加载,处理等工作
    def __init__(self):
        pass

    #定义数据的条数
    def __len__(self):
        return 500

    #根据序号i,获取一条数据
    def __getitem__(self, i):
        #获取一个矩形的数据
        width, height, s = get_rectangle()

        #定义宽高为x,定义面积为y
        x = torch.FloatTensor([width, height])
        y = torch.FloatTensor([s])

        return x, y


dataset = Dataset()

len(dataset), dataset[0]

(500, (tensor([0.4518, 0.0109]), tensor([0.0049])))

In [3]:
#数据集加载器,每8条数据为一个批次,打乱顺序,不足8条时丢弃尾数
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=8,
                                     shuffle=True,
                                     drop_last=True)

len(loader), next(iter(loader))

(62,
 [tensor([[0.3174, 0.8914],
          [0.0360, 0.5670],
          [0.1893, 0.5694],
          [0.6303, 0.6357],
          [0.3546, 0.4646],
          [0.8690, 0.9435],
          [0.6486, 0.0783],
          [0.8166, 0.8570]]),
  tensor([[0.2829],
          [0.0204],
          [0.1078],
          [0.4007],
          [0.1648],
          [0.8199],
          [0.0508],
          [0.6998]])])

In [4]:
#全连接神经网络
class Model(torch.nn.Module):

    #模型初始化部分
    def __init__(self):
        super().__init__()

        #定义神经网络结构
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(in_features=2, out_features=32),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=32, out_features=32),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=32, out_features=1),
        )

    #定义神经网络计算过程
    def forward(self, x):
        return self.fc(x)


model = Model()

model(torch.randn(8, 2)).shape

torch.Size([8, 1])

In [5]:
#训练
def train():
    #优化器,根据梯度调整模型参数
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    #计算loss的函数
    loss_fun = torch.nn.MSELoss()

    #让model进入train模式,开启dropout等功能
    model.train()

    #全量数据遍历100轮
    for epoch in range(100):

        #按批次遍历loader中的数据
        for i, (x, y) in enumerate(loader):

            #模型计算
            out = model(x)

            #根据计算结果和y的差,计算loss,在计算结果完全正确的情况下,loss为0
            loss = loss_fun(out, y)

            #根据loss计算模型的梯度
            loss.backward()

            #根据梯度调整模型的参数
            optimizer.step()

            #梯度归零,准备下一轮的计算
            optimizer.zero_grad()

        if epoch % 20 == 0:
            print(epoch, loss.item())
            
    #保存模型到磁盘
    torch.save(model, 'model/4.model')


train()

0 0.15631675720214844
20 0.0009868389461189508
40 0.0007930236752144992
60 0.0002548572374507785
80 4.957337296218611e-05


In [6]:
#测试
#注释的表明不计算模型梯度,节省计算资源
@torch.no_grad()
def test():
    
    #从磁盘加载模型
    model = torch.load('model/4.model')

    #模型进入测试模式,关闭dropout等功能
    model.eval()

    #获取一批数据
    x, y = next(iter(loader))

    #模型计算结果
    out = model(x)

    print(torch.cat([out, y], dim=1))


test()

tensor([[0.0931, 0.0875],
        [0.1399, 0.1377],
        [0.6393, 0.6347],
        [0.0492, 0.0516],
        [0.2065, 0.2130],
        [0.3257, 0.3235],
        [0.2503, 0.2601],
        [0.7016, 0.7113]])
